In [1]:
import pandas as pd
import re

In [55]:
meta_data = 'meta_data.json'
df = pd.read_json(meta_data)

In [136]:
df

,database,url,title,authors,journal,citations,publish_date,keywords,doi,journal_impact_factor
0,ieeexplore,https://ieeexplore.ieee.org/document/6027518,A visualization and modeling tool for security...,"[Reijo M. Savola ;, Petri Heinonen]",2011 Information Security for South Africa,10.0,2011,"[Measurement, Visualization, Authentication, D...",10.1109/ISSA.2011.6027518,NaN
1,sciencedirect,https://www.sciencedirect.com/science/article/...,Information security requirements – Interpreti...,"[{'surname': 'Gerber', 'first_name': 'Mariana'...",Computers & Security,21.0,2008,"[Information security, Information security ma...",https://doi.org/10.1016/j.cose.2008.07.009,3.062
2,ieeexplore,https://ieeexplore.ieee.org/document/7166125,Cyber security operations centre: Security mon...,[Cyril Onwubiko],2015 International Conference on Cyber Situati...,6.0,2015,"[Monitoring, Business, Computer crime, System-...",10.1109/CyberSA.2015.7166125,NaN
3,sciencedirect,https://www.sciencedirect.com/science/article/...,Security Management Standards: A Mapping,"[{'surname': 'Haufe', 'first_name': 'Knut'}, {...",Procedia Computer Science,NaN,2016,"[Information Security, ISMS, Mapping, Processe...",https://doi.org/10.1016/j.procs.2016.09.221,NaN
4,sciencedirect,https://www.sciencedirect.com/science/article/...,Mapping the field of software life cycle secur...,"[{'surname': 'Morrison', 'first_name': 'Patric...",Information and Software Technology,3.0,2018,"[Metrics, Measurement, Security]",https://doi.org/10.1016/j.infsof.2018.05.011,2.921
...,...,...,...,...,...,...,...,...,...,...
137,sciencedirect,https://www.sciencedirect.com/science/article/...,A survey of security issues for cloud computing,"[{'surname': 'Khan', 'first_name': 'Minhaj Ahm...",Journal of Network and Computer Applications,77.0,2016,"[Cloud security, Cloud computing, Denial-of-se...",https://doi.org/10.1016/j.jnca.2016.05.010,5.273
138,ieeexplore,https://ieeexplore.ieee.org/document/7921989,A framework for measurability of security,"[Seraj Fayyad ;, Josef Noll]",2017 8th International Conference on Informati...,1.0,2017,"[Security, Measurement, Databases, Privacy, Sy...",10.1109/IACS.2017.7921989,NaN
139,ieeexplore,https://ieeexplore.ieee.org/document/5467019,A Formal Approach Enabling Risk-Aware Business...,"[Simon Tjoa ;, Stefan Jakoubi ;, Gernot Goluch...",IEEE Transactions on Services Computing,15.0,2010,"[Security, Unified modeling language, Risk man...",10.1109/TSC.2010.17,5.707
140,ieeexplore,https://ieeexplore.ieee.org/document/7367050,The effectiveness of corporate culture toward ...,"[Abdullah Almubark ;, Nobutoshi Hatanaka ;, Os...",2015 World Congress on Information Technology ...,133.0,2015,"[Decision support systems, Information securit...",10.1109/WCITCA.2015.7367050,NaN


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142 entries, 0 to 141
Data columns (total 10 columns):
database                 142 non-null object
url                      142 non-null object
title                    142 non-null object
authors                  142 non-null object
journal                  142 non-null object
citations                135 non-null float64
publish_date             140 non-null object
keywords                 142 non-null object
doi                      139 non-null object
journal_impact_factor    51 non-null float64
dtypes: float64(2), object(8)
memory usage: 11.2+ KB


In [58]:
# Optimizes the database column to categorys
df["database"] = df["database"].astype('category')

# Reassigned the publish date
df["publish_date"][1] = 2008
df["publish_date"][6] = 2018
df["publish_date"][17] = 2015
df["publish_date"][20] = 2018
df["publish_date"][62] = 2018
df["publish_date"][25] = 2013
df["publish_date"][31] = 2017
df["publish_date"][39] = 2010
df["publish_date"][53] = 2019
df["publish_date"][74] = 2017
df["publish_date"][80] = 2010
df["publish_date"][89] = 2018
df["publish_date"][110] = 2012
df["publish_date"][135] = 2017

In [129]:
type(df["publish_date"].unique()[24])

NoneType

In [111]:
df["url"][135]

'https://www.sciencedirect.com/science/article/pii/S2211368117300207'

In [135]:
# Corrections
df.loc[df["publish_date"] == df["publish_date"].unique()[24]]

,database,url,title,authors,journal,citations,publish_date,keywords,doi,journal_impact_factor
